In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('churn.csv')

In [ ]:
df.head(5)

In [ ]:
# Data Dictionary --> Details about the data/about each features

In [ ]:
# EDA & Data Cleaning

In [ ]:
# Univariate & Multi-variate analysis --> Measures of Central tendency, dispersion & shape

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
round(df.describe(),2).T

In [ ]:
df.head(5)

In [ ]:
df['Geography'].unique()

In [ ]:
df['Geography'].value_counts()

In [ ]:
import numpy as np

In [ ]:
df.columns

In [ ]:
df.groupby('Geography').agg({'EstimatedSalary':[np.mean,np.std],
                             'Exited':[np.mean,'count'],'Balance':['min','max',np.mean]})

###### Call of concern : German (most premium custs) are leaving

In [ ]:
df.head()

In [ ]:
# Do a detailed analysis/cleaning & visualisation of the data

In [ ]:
# Class distribution for Classification

In [ ]:
# Distribution of the target variable --> Regression

In [ ]:
df['Exited'].value_counts()

In [ ]:
# Baseline accuracy

In [ ]:
7963/10000

In [ ]:
2037/10000

In [ ]:
#!pip install --user seaborn

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(df['CreditScore'])

In [ ]:
sns.boxplot(y=df['CreditScore'])

In [ ]:
df.head()

In [ ]:
x = df.iloc[:,[2,5,6,7,8,9,10,11]]

In [ ]:
# One hot, label encoding --> numpy arrays

In [ ]:
dummies = pd.get_dummies(df[['Geography','Gender']], drop_first=True)

In [ ]:
x = pd.concat([x,dummies], axis=1)

In [ ]:
y=df['Exited']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
1617/2000

In [ ]:
6346/8000

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lg = LogisticRegression()

In [ ]:
model_lg = lg.fit(x_train,y_train)

In [ ]:
prediction = model_lg.predict(x_test)

In [ ]:
pd.crosstab(y_test,prediction)

In [ ]:
# Accuracy
(1584+15)/2000

In [ ]:
# Precision

In [ ]:
15/(15+33)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtree = DecisionTreeClassifier(min_samples_leaf=10,min_samples_split=5, max_depth=4)

In [ ]:
model_dt = dtree.fit(x_train,y_train)

In [ ]:
pred_dtree = model_dt.predict(x_test)

In [ ]:
pd.crosstab(y_test,pred_dtree)

In [ ]:
(1414+174)/2000

In [ ]:
174/(174+204)

In [ ]:
(1560+137)/2000

In [ ]:
137/(137+57)

In [ ]:
# Explore Ensemble models for better results

In [ ]:
!whereis spark

In [ ]:
import findspark

In [ ]:
findspark.init('/usr/local/spark')

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("classifier").getOrCreate()

In [ ]:
spark

In [ ]:
df1 = spark.read.csv('churn.csv', header=True, inferSchema=True)

In [ ]:
df1.head()

In [ ]:
df1.printSchema()

In [ ]:
# EDA, cleaning & detailed analysis

In [ ]:
# df1.toPandas

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, 

In [ ]:
cat_id = StringIndexer(inputCol="Geography", outputCol="Geo").fit(df1)

In [ ]:
df1 = cat_id.transform(df1)

In [ ]:
df1.select('Geo','Geography').show()

In [ ]:
cat_id1 = StringIndexer(inputCol="Gender", outputCol="Gen").fit(df1)

In [ ]:
df1 = cat_id1.transform(df1)

In [ ]:
df1.select('Gen','Gender').show(15)

In [ ]:
cat_one = OneHotEncoder(inputCol="Geo",outputCol="Geon_encoded")

In [ ]:
df1=cat_one.transform(df1)

In [ ]:
df1.select('Geo','Geography',"Geon_encoded").show(15)

In [ ]:
cat_two = OneHotEncoder(inputCol="Gen",outputCol="Gen_encoded")

In [ ]:
df1=cat_two.transform(df1)

In [ ]:
# from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
df.columns

In [ ]:
df_assembler = VectorAssembler(inputCols=['CreditScore','Age','Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary','Geon_encoded','Gen_encoded'], outputCol="features")

In [ ]:
df1 = df_assembler.transform(df1)

In [ ]:
df1.select("features").show(5)

In [ ]:
final_df = df1.select("features","Exited")

In [ ]:
final_df.show(5)

In [ ]:
train_data,test_data = final_df.randomSplit([0.8,0.2], seed=42)

In [ ]:
train_data.count()

In [ ]:
test_data.count()

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier, LogisticRegression

In [ ]:
lg1 = LogisticRegression(labelCol="Exited")

In [ ]:
model_lg1 = lg1.fit(train_data)

In [ ]:
pred_lg1 = model_lg1.evaluate(test_data)

In [ ]:
pred_lg1.predictions.show(5)

In [ ]:
pred_lg1.predictions.collect()

In [ ]:
pred_lg2 = model_lg1.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [ ]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Exited')

In [ ]:
eval.evaluate(pred_lg2)

In [ ]:
dtree = DecisionTreeClassifier(featuresCol='features', labelCol='Exited')

In [ ]:
model_tree = dtree.fit(train_data)

In [ ]:
pred_tree=model_tree.transform(test_data)

In [ ]:
pred_tree.show()

In [ ]:
eval_tree = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='Exited')

In [ ]:
acc = eval_tree.evaluate(pred_tree)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
y_origin = pred_tree.select('Exited').collect()

In [ ]:
y_predicted = pred_tree.select('prediction').collect()

In [ ]:
cm = confusion_matrix(y_true=y_origin,y_pred=y_predicted)

In [ ]:
# Accuracy
(187+1465)/2000

In [ ]:
# Precision
187/(187+88)